<a href="https://colab.research.google.com/github/shehuphd/aqc-hackathon/blob/main/aqc_finance_Shehu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cell 1 — data loading and standardisation (data-first, dataset-agnostic)

In this cell, we load a CSV dataset, enforce a strict structure, clean the data, and prepare it for downstream use. We start a timer at the top of the cell to track how long the entire process takes, which becomes useful as datasets grow or when we rerun the notebook frequently.

We import pandas to load and manipulate tabular data, and NumPy for numerical operations that may be needed later. There is a single variable for the dataset path, and this is the only line to change when switching datasets.

Before loading anything, we define a required schema. This is a strict list of column names that must exist in the CSV. We then load the dataset from disk and print the column names so we can immediately see what was read. The code checks for missing required columns, and if any one is absent, it raises an error and stops execution. This forces schema correctness early and prevents subtle bugs later in the notebook.

Once the schema checks pass, we standardise the column names to lowercase, snake_case versions. We then coerce each column into its expected data type:
- Asset names and sectors are treated as strings and trimmed of extra whitespace
- Empty sector values replaced by a placeholder.
- Numeric fields are explicitly converted to numbers.
- Any invalid values are coerced into missing values.

After type coercion, the code cleans missing values using sensible defaults:
- Missing expected returns and transaction costs are treated as zero
- Missing market capitalisation values are filled with the median market cap, which avoids skewing the data with extreme assumptions
- Missing previous position values are set to zero and converted to integers, reflecting that the asset was not previously held.

A few sanity checks are then applied to ensure logical consistency:
- We verify that each asset appears only once,
- That previous positions are strictly binary values, and
- That the dataset is not empty after loading.

Any violation throws an error.

Finally, the cell prints a short summary confirming successful loading, including the number of assets, the number of unique sectors, and the final internal column names. The runtime of the cell is displayed, and the first ten rows of the cleaned dataset are shown for a quick visual check.

In [15]:
# ==============================
# Cell 1: Load and standardise data (strict schema)
# ==============================

import time
start_time = time.perf_counter()

import pandas as pd
import numpy as np
import math

# ---- CONFIG: change only this path ----
DATA_PATH = "/content/drive/MyDrive/colab_notebooks/Finance/aqc_dataset.csv"

# ---- REQUIRED SCHEMA (dataset must contain these exact columns) ----
REQUIRED_COLUMNS = [
    "Asset",
    "Sector",
    "Expected_Return",
    "Previous_Position",
    "Transaction_Cost",
    "Market_Cap",
]

# ---- LOAD & DISPLAY ----
df_raw = pd.read_csv(DATA_PATH)
print(df_raw.columns.tolist())

# ---- VALIDATE REQUIRED COLUMNS ----
missing = [c for c in REQUIRED_COLUMNS if c not in df_raw.columns]
if missing:
    raise ValueError(
        "Dataset schema mismatch. Missing required columns: "
        + ", ".join(missing)
    )

# ---- STANDARDISE COLUMN NAMES ----
df = df_raw.rename(columns={
    "Asset": "asset",
    "Sector": "sector",
    "Expected_Return": "expected_return",
    "Previous_Position": "previous_position",
    "Transaction_Cost": "transaction_cost",
    "Market_Cap": "market_cap",
}).copy()

# ---- COERCE TYPES ----
df["asset"] = df["asset"].astype(str).str.strip()
df["sector"] = df["sector"].astype(str).str.strip().replace({"": "UNKNOWN"})

df["expected_return"] = pd.to_numeric(df["expected_return"], errors="coerce")
df["transaction_cost"] = pd.to_numeric(df["transaction_cost"], errors="coerce")
df["market_cap"] = pd.to_numeric(df["market_cap"], errors="coerce")
df["previous_position"] = pd.to_numeric(df["previous_position"], errors="coerce")

# ---- CLEAN / DEFAULTS ----
df["expected_return"] = df["expected_return"].fillna(0.0)
df["transaction_cost"] = df["transaction_cost"].fillna(0.0)

if df["market_cap"].isna().any():
    df["market_cap"] = df["market_cap"].fillna(df["market_cap"].median())

df["previous_position"] = df["previous_position"].fillna(0).astype(int)

# ---- SANITY CHECKS ----
if df["asset"].duplicated().any():
    dupes = df.loc[df["asset"].duplicated(), "asset"].unique().tolist()
    raise ValueError(f"Duplicate assets found: {dupes[:10]}")

bad_prev = sorted(set(df["previous_position"].unique()) - {0, 1})
if bad_prev:
    raise ValueError(f"previous_position must be 0 or 1. Found: {bad_prev}")

if len(df) == 0:
    raise ValueError("Dataset is empty after loading.")

# ---- SUMMARY ----
print("Dataset loaded and standardised successfully.")
print(f"Number of assets: {len(df)}")
print(f"Sectors: {df['sector'].nunique()}")
print("Internal columns:", df.columns.tolist())

# ---- TIMING ----
elapsed = time.perf_counter() - start_time
print(f"\nCell 1 runtime: {elapsed:.3f} seconds\n")

df.head(10)


['Asset', 'Sector', 'Expected_Return', 'Previous_Position', 'Transaction_Cost', 'Market_Cap']
Dataset loaded and standardised successfully.
Number of assets: 50
Sectors: 5
Internal columns: ['asset', 'sector', 'expected_return', 'previous_position', 'transaction_cost', 'market_cap']

Cell 1 runtime: 0.032 seconds



,asset,sector,expected_return,previous_position,transaction_cost,market_cap
0,ASSET_003,CONS,0.386559,0,0.044633,35.345530
1,ASSET_007,HEALTH,-0.152227,0,0.040831,8.463425
2,ASSET_009,CONS,0.023442,0,0.044358,1.908734
3,ASSET_010,ENERGY,-0.030733,0,0.017441,5.691906
4,ASSET_019,ENERGY,1.528005,0,0.040161,16.221097
5,ASSET_022,ENERGY,0.430408,0,0.047232,12.426917
6,ASSET_032,ENERGY,0.665292,0,0.036675,8.781033
7,ASSET_033,TECH,0.538884,0,0.042932,4.745871
8,ASSET_038,TECH,0.115897,0,0.019676,26.895169
9,ASSET_039,FIN,0.782384,0,0.046443,28.469388


## Cell 2 — config + multi-start portfolio generation (N is variable)

In this cell, we define the configuration for portfolio construction and generate multiple “starting portfolios” using different heuristics. The idea is to begin optimization from several sensible but distinct starting points, rather than committing to a single worldview upfront. We start a timer again so we can track how long this setup step takes.

We first import a few utilities. The random module is used for stochastic starts, display lets us render clean tables in Colab, and some standard collections help with counting and grouping later. We then define a single config dictionary. This is where we control portfolio size, sector concentration limits, how many random starts to generate, and whether randomness should be reproducible. For debugging, we can fix the random seed; otherwise, each run produces genuinely different random portfolios.

Based on that configuration, we explicitly seed the random number generator. When `use_fixed_seed` is on, runs are deterministic. When it is off, every run explores a slightly different part of the solution space.

The next section defines a set of small, focused helper functions. Each function generates a portfolio using a different logic:
- Some are deterministic and transparent, such as selecting the top assets by expected return or by market capitalisation.
- Others bias toward practical constraints, such as sticking close to previous holdings to reduce turnover.
- There are also diversification-oriented approaches, including sector-aware selection and randomised selection with soft sector caps to avoid extreme concentration.

Together, these functions encode different intuitions about what a “reasonable” starting portfolio might look like.

We also define helper utilities for summarising and inspecting these starts.
- One function aggregates high-level statistics for a given start, including total and average expected return, sector distribution, average market cap, and how many assets were carried over from the previous portfolio.
- Another function allows us to inspect the full asset-level breakdown for any specific start, which is useful for debugging or qualitative review.

With the helpers in place, we build the actual starts. We fix the target portfolio size from the config, then generate a dictionary of named starting portfolios. Each entry represents a different philosophy: return-first, safety-first, diversification-first, low-turnover-first, and several random but constrained alternatives. We also optionally generate multiple independent random starts to further increase diversity.

Once all starts are generated, we convert their summaries into a table. We round key metrics for readability and print a short configuration recap so it is always clear how these portfolios were generated. We then stop the timer and report the runtime for the cell.

Finally, we sort the summary table by total expected return and display it as a simple, transparent baseline ranking of all starting portfolios. Optionally, if a specific start name is provided in the config, we display a detailed asset-level view of that portfolio.

In [16]:
# ==============================
# Cell 2: Config + multi-start initial portfolios (clean table output)
# ==============================

start_time = time.perf_counter()

import random
from IPython.display import display
from collections import Counter, defaultdict

# ---- CONFIG (tune these freely) ----
config = {
    "n_target": 40,                 # initially tied to the total number of portfolio assets; start small
    "sector_cap": 5,                # can later be 7 or more
    "max_changes_preferred": 10,    # soft preference (not enforced yet)

    # randomness control
    "use_fixed_seed": False,        # ← switch for debugging purposes
    "random_seed": 42,

    # start generation knobs
    "random_starts": 3,             # how many random starts to generate
    "diverse_first_per_sector": 1,  # diversification start: take this many per sector first

    # output knobs
    "show_detail_for": None,        # e.g. "Start 1: Return-first" to display a detailed table
}

if config.get("use_fixed_seed", False):
    random.seed(config["random_seed"])
else:
    random.seed(None)  # true randomness every run

# ---- HELPERS ----
def top_by_expected_return(df, n):
    return df.sort_values("expected_return", ascending=False).head(n)["asset"].tolist()

def top_by_market_cap(df, n):
    return df.sort_values("market_cap", ascending=False).head(n)["asset"].tolist()

def previous_holdings(df):
    return df.loc[df["previous_position"] == 1, "asset"].tolist()

def diversify_first(df, n, per_sector_first=1):
    # Take a small number from each sector first (highest expected return within that sector),
    # then fill remaining slots by expected return.
    chosen = []
    df_sorted = df.sort_values("expected_return", ascending=False)

    for sector, group in df_sorted.groupby("sector"):
        picks = group.head(per_sector_first)["asset"].tolist()
        for a in picks:
            if len(chosen) < n and a not in chosen:
                chosen.append(a)

    for a in df_sorted["asset"].tolist():
        if len(chosen) >= n:
            break
        if a not in chosen:
            chosen.append(a)

    return chosen[:n]

def diversify_random(df, n, sector_cap=None):
    # Randomly pick assets while spreading across sectors.
    # If sector_cap is provided, it avoids extreme concentration.
    sector_map = df.set_index("asset")["sector"].to_dict()
    assets_by_sector = df.groupby("sector")["asset"].apply(list).to_dict()

    chosen = []
    counts = defaultdict(int)

    # First pass: try to take at least 1 from each sector (random)
    sectors = list(assets_by_sector.keys())
    random.shuffle(sectors)

    for s in sectors:
        if len(chosen) >= n:
            break
        picks = assets_by_sector[s][:]
        random.shuffle(picks)
        for a in picks:
            if len(chosen) >= n:
                break
            if a in chosen:
                continue
            if sector_cap is not None and counts[s] >= sector_cap:
                break
            chosen.append(a)
            counts[s] += 1
            break

    # Fill remaining completely at random (optionally respecting sector cap)
    remaining = [a for a in df["asset"].tolist() if a not in chosen]
    random.shuffle(remaining)

    for a in remaining:
        if len(chosen) >= n:
            break
        s = sector_map[a]
        if sector_cap is not None and counts[s] >= sector_cap:
            continue
        chosen.append(a)
        counts[s] += 1

    return chosen[:n]

def low_turnover_random(df, n):
    prev = df.loc[df["previous_position"] == 1, "asset"].tolist()
    rest = df.loc[df["previous_position"] == 0, "asset"].tolist()

    random.shuffle(prev)
    chosen = prev[:n]

    if len(chosen) < n:
        random.shuffle(rest)
        chosen += rest[: (n - len(chosen))]

    return chosen[:n]

def low_turnover_start(df, n):
    prev = previous_holdings(df)

    # If we already hold >= n, keep the best n of those by expected return
    if len(prev) >= n:
        sub = df[df["asset"].isin(prev)].sort_values("expected_return", ascending=False)
        return sub.head(n)["asset"].tolist()

    # Otherwise, start with what we have, then top up with best expected return
    chosen = list(prev)
    df_sorted = df.sort_values("expected_return", ascending=False)
    for a in df_sorted["asset"].tolist():
        if len(chosen) >= n:
            break
        if a not in chosen:
            chosen.append(a)
    return chosen[:n]

def random_sane_start(df, n, sector_cap):
    # Random picks, but avoid extreme sector concentration using a simple cap.
    assets = df["asset"].tolist()
    random.shuffle(assets)

    chosen = []
    counts = defaultdict(int)
    sector_map = df.set_index("asset")["sector"].to_dict()

    for a in assets:
        if len(chosen) >= n:
            break
        s = sector_map[a]
        if counts[s] < sector_cap:
            chosen.append(a)
            counts[s] += 1

    # If we couldn't fill (rare), fill remaining ignoring cap
    if len(chosen) < n:
        for a in assets:
            if len(chosen) >= n:
                break
            if a not in chosen:
                chosen.append(a)

    return chosen[:n]

def summarise_start(df, name, assets):
    sub = df[df["asset"].isin(assets)]
    sector_counts = sub["sector"].value_counts().to_dict()
    sector_str = ", ".join([f"{k}:{v}" for k, v in sector_counts.items()])

    return {
        "start_name": name,
        "num_assets": len(assets),
        "sectors": sector_str,
        "total_expected_return": float(sub["expected_return"].sum()),
        "avg_expected_return": float(sub["expected_return"].mean()),
        "avg_market_cap": float(sub["market_cap"].mean()),
        "held_from_previous": int(sub["previous_position"].sum()),
    }

def inspect_start(df, starts, start_name):
    assets = starts[start_name]
    sub = df[df["asset"].isin(assets)].sort_values("expected_return", ascending=False)
    display(sub[[
        "asset",
        "sector",
        "expected_return",
        "market_cap",
        "transaction_cost",
        "previous_position"
    ]])

# ---- BUILD STARTS ----
N = config["n_target"]
starts = {}

starts["Return-first"] = top_by_expected_return(df, N)
starts["Safety-first (market cap tilt)"] = top_by_market_cap(df, N)
starts["Diversification-first"] = diversify_first(df, N, config["diverse_first_per_sector"])
starts["Low-turnover-first (stick close to previous)"] = low_turnover_start(df, N)
starts["Diversification (random)"] = diversify_random(df, N, sector_cap=config["sector_cap"])
starts["Low turnover (random)"] = low_turnover_random(df, N)

for i in range(config["random_starts"]):
    starts[f"R{i+1}: Random"] = random_sane_start(df, N, config["sector_cap"])

# ---- CLEAN TABLE OUTPUT ----
summary_df = pd.DataFrame([summarise_start(df, name, assets) for name, assets in starts.items()])

# Nicer formatting for display
summary_df["total_expected_return"] = summary_df["total_expected_return"].round(4)
summary_df["avg_expected_return"] = summary_df["avg_expected_return"].round(4)
summary_df["avg_market_cap"] = summary_df["avg_market_cap"].round(2)

print("Multi-start portfolios generated, sorted by expected return.\n")
print(f"Number of targets: {config['n_target']} \nSector cap: {config['sector_cap']} \nRandom starts: {config['random_starts']}\n")

# ---- TIMING ----
elapsed = time.perf_counter() - start_time
print(f"\nCell 2 runtime: {elapsed:.3f} seconds\n")

# Sort by total expected return (simple, transparent baseline ranking)
summary_df = summary_df.sort_values("total_expected_return", ascending=False).reset_index(drop=True)
display(summary_df)

# ---- OPTIONAL DETAIL VIEW ----
if config["show_detail_for"] is not None:
    if config["show_detail_for"] not in starts:
        raise ValueError(f"Unknown start_name in show_detail_for. Available: {list(starts.keys())}")
    inspect_start(df, starts, config["show_detail_for"])


Multi-start portfolios generated, sorted by expected return.

Number of targets: 40 
Sector cap: 5 
Random starts: 3


Cell 2 runtime: 0.029 seconds



,start_name,num_assets,sectors,total_expected_return,avg_expected_return,avg_market_cap,held_from_previous
0,Return-first,40,"ENERGY:12, CONS:9, HEALTH:7, TECH:6, FIN:6",16.5197,0.4130,15.54,27
1,Diversification-first,40,"ENERGY:12, CONS:9, HEALTH:7, TECH:6, FIN:6",16.5197,0.4130,15.54,27
2,Low-turnover-first (stick close to previous),40,"ENERGY:10, FIN:9, HEALTH:9, CONS:8, TECH:4",11.0423,0.2761,14.72,36
3,Safety-first (market cap tilt),40,"ENERGY:12, FIN:8, HEALTH:7, TECH:7, CONS:6",9.8611,0.2465,17.11,28
4,R1: Random,40,"ENERGY:11, CONS:9, FIN:8, HEALTH:7, TECH:5",9.7994,0.2450,15.28,30
5,Low turnover (random),40,"ENERGY:10, CONS:9, HEALTH:9, FIN:7, TECH:5",9.5041,0.2376,14.93,36
6,R2: Random,40,"ENERGY:9, HEALTH:9, CONS:8, TECH:7, FIN:7",7.6676,0.1917,11.81,32
7,Diversification (random),25,"CONS:5, HEALTH:5, ENERGY:5, TECH:5, FIN:5",7.1445,0.2858,19.53,15
8,R3: Random,40,"ENERGY:12, HEALTH:9, CONS:7, FIN:7, TECH:5",6.7045,0.1676,13.16,31


## Cell 3 — score starts and select best starting portfolio

In this cell, we take the candidate starting portfolios from the previous step, score them using a single composite objective, and select the best one to use as our optimization starting point. As before, we start a timer so we can track how long this evaluation step takes.

We begin by printing the list of available starts and a small sample from the first one. This acts as a quick sanity check to confirm that the inputs look reasonable before scoring begins. We then import the libraries needed for numerical operations, tabular aggregation, and counting.

Next, we define a scoring configuration. This pulls in shared constraints from the earlier config, such as target portfolio size, soft sector caps, and a preferred maximum number of changes. We also define weights that control how expected return trades off against risk proxies, transaction costs, sector concentration penalties, and turnover penalties. Adjusting these weights directly changes how starts are ranked.

We then define helper functions that break the scoring logic into small, readable pieces. Some functions compute structural properties, such as sector counts and concentration. Others approximate risk using average market capitalisation and sector concentration. We also explicitly compute buys and sells relative to the previous portfolio so that turnover and transaction costs are measured directly.

The scoring function combines all of this. For a given start, it computes total expected return as the reward term, then subtracts penalties for risk, transaction costs, sector concentration beyond a soft cap, and turnover beyond a preferred limit. Sector and turnover penalties are applied non-linearly, so small violations are tolerated while larger ones are penalised more heavily.

With the scoring logic in place, we score every starting portfolio. For each start, we compute the score and its components, store a readable sector breakdown, and collect everything into a table. The table is sorted by total score. This ranks all candidate starts.

Finally, we select the highest-scoring start (even if it's negative) and extract its asset list as the current working portfolio. This becomes the baseline for the optimization or search step that follows. We print the chosen start name, stop the timer, and report the runtime for the cell.

## How to read the scores and why they are negative

The table produced by this cell is ranked only by total_score. The start in the first row after sorting is always the one selected. No other column is used for selection. Higher total_score is better, even when all values are negative.

Scores are often negative because the objective function is built as reward minus penalties, not as a normalised score. Expected return contributes positively, but several penalty terms subtract from it: risk proxies, transaction costs, sector concentration beyond the soft cap, and turnover beyond the preferred limit. The sector and turnover penalties are applied non-linearly, so large violations quickly dominate the score.

As a result, a start with very high expected return can still score poorly overall if it makes many changes, concentrates heavily in a few sectors, or incurs high transaction costs. Conversely, a start with more modest returns can rank higher if it balances return with lower risk, lower turnover, and better diversification.

In short, the absolute value of the score is less important than the ranking. The chosen start is simply the one with the highest composite score, meaning it offers the best trade-off across all objectives under the current weights and constraints.

In [17]:
# ==============================
# Cell 3: Score starts and select best starting portfolio
# ==============================

start_time = time.perf_counter()

print("Starts being scored:", list(starts.keys()))
print("First start sample:", list(starts.values())[0][:5])

from collections import Counter

# ---- SCORING CONFIG (tune freely) ----
score_config = {
    "n_target": config["n_target"],
    "sector_cap_soft": config["sector_cap"],
    "max_changes_soft": config["max_changes_preferred"],

    "w_return": 1.0,
    "w_risk": 0.8,
    "w_txn": 1.0,
    "w_sector_pain": 2.0,
    "w_change_pain": 2.5,
    "pain_power": 2.0,
}

# ---- HELPERS ----
def sector_counts(df, assets):
    return df[df["asset"].isin(assets)]["sector"].value_counts().to_dict()

def market_cap_risk(df, assets):
    avg_cap = df[df["asset"].isin(assets)]["market_cap"].mean()
    return 1.0 / max(avg_cap, 1e-6)

def concentration_risk(df, assets):
    counts = sector_counts(df, assets)
    n = len(assets)
    return sum((c / n) ** 2 for c in counts.values())

def compute_changes(prev_assets, assets):
    prev, curr = set(prev_assets), set(assets)
    return list(curr - prev), list(prev - curr)

def transaction_cost(df, buys, sells):
    return df[df["asset"].isin(buys + sells)]["transaction_cost"].sum()

def score_start(df, assets, prev_assets, cfg):
    sub = df[df["asset"].isin(assets)]

    reward = sub["expected_return"].sum()
    risk = market_cap_risk(df, assets) + concentration_risk(df, assets)

    buys, sells = compute_changes(prev_assets, assets)
    txn = transaction_cost(df, buys, sells)

    sector_over = sum(
        max(0, c - cfg["sector_cap_soft"])
        for c in sector_counts(df, assets).values()
    )

    change_over = max(0, len(buys) + len(sells) - cfg["max_changes_soft"])

    total = (
        cfg["w_return"] * reward
        - cfg["w_risk"] * risk
        - cfg["w_txn"] * txn
        - cfg["w_sector_pain"] * (sector_over ** cfg["pain_power"])
        - cfg["w_change_pain"] * (change_over ** cfg["pain_power"])
    )

    return {
        "total_score": total,
        "total_expected_return": reward,
        "risk_proxy": risk,
        "transaction_cost": txn,
        "num_changes": len(buys) + len(sells),
        "sector_counts": sector_counts(df, assets),
    }

# ---- SCORE ALL STARTS ----
prev_assets = df.loc[df["previous_position"] == 1, "asset"].tolist()

rows = []
for name, assets in starts.items():
    s = score_start(df, assets, prev_assets, score_config)
    rows.append({
        "start_name": name,
        **{k: v for k, v in s.items() if k != "sector_counts"},
        "sectors": ", ".join(f"{k}:{v}" for k, v in s["sector_counts"].items()),
    })

start_scores_df = (
    pd.DataFrame(rows)
    .sort_values("total_score", ascending=False)
    .reset_index(drop=True)
)

display(start_scores_df)

best_start_name = start_scores_df.loc[0, "start_name"]
current_assets = starts[best_start_name]

print(f"Chosen start: {best_start_name}")

# ---- TIMING ----
elapsed = time.perf_counter() - start_time
print(f"\nCell 3 runtime: {elapsed:.3f} seconds\n")


Starts being scored: ['Return-first', 'Safety-first (market cap tilt)', 'Diversification-first', 'Low-turnover-first (stick close to previous)', 'Diversification (random)', 'Low turnover (random)', 'R1: Random', 'R2: Random', 'R3: Random']
First start sample: ['ASSET_019', 'ASSET_021', 'ASSET_043', 'ASSET_042', 'ASSET_013']


,start_name,total_score,total_expected_return,risk_proxy,transaction_cost,num_changes,sectors
0,Low turnover (random),-440.877344,9.504087,0.276984,0.159844,4,"ENERGY:10, CONS:9, HEALTH:9, FIN:7, TECH:5"
1,R2: Random,-453.011342,7.667647,0.287142,0.449276,12,"ENERGY:9, HEALTH:9, CONS:8, TECH:7, FIN:7"
2,R3: Random,-484.053083,6.704502,0.293468,0.522811,14,"ENERGY:12, HEALTH:9, CONS:7, FIN:7, TECH:5"
3,Low-turnover-first (stick close to previous),-501.328882,11.042289,0.281696,0.145814,4,"ENERGY:10, FIN:9, HEALTH:9, CONS:8, TECH:4"
4,R1: Random,-531.009839,9.799376,0.277954,0.586852,16,"ENERGY:11, CONS:9, FIN:8, HEALTH:7, TECH:5"
5,Safety-first (market cap tilt),-691.036373,9.861100,0.272208,0.679706,20,"ENERGY:12, FIN:8, HEALTH:7, TECH:7, CONS:6"
6,Return-first,-794.419952,16.519656,0.280619,0.715113,22,"ENERGY:12, CONS:9, HEALTH:7, TECH:6, FIN:6"
7,Diversification-first,-794.419952,16.519656,0.280619,0.715113,22,"ENERGY:12, CONS:9, HEALTH:7, TECH:6, FIN:6"
8,Diversification (random),-1096.515033,7.144505,0.251209,0.958570,31,"CONS:5, HEALTH:5, ENERGY:5, TECH:5, FIN:5"


Chosen start: Low turnover (random)

Cell 3 runtime: 0.067 seconds



## Cell 4 — optimization cycles and final portfolio output

In this cell, we take the starting portfolio we selected in Cell 3 (`current_assets`) and try to improve it through a simple repeated search process. We start a timer at the top so we can track runtime, and we keep the whole run explainable by recording what happens in each cycle.

We set up a random number generator that respects the same `use_fixed_seed` switch from earlier. If we fix the seed, we can reproduce results exactly for debugging. If we do not, each run explores different swap paths, which helps when we want to see if we can find a better portfolio from the same starting point.

The optimization move we allow is deliberately simple: a one-out, one-in swap. The helper function `propose_swap` takes the current portfolio, randomly chooses one held asset to remove, randomly chooses one not-held asset to add, and returns the new candidate portfolio. This keeps the portfolio size constant at `N` and makes every candidate easy to explain as a single swap.

We then decide how long to run the optimization. Instead of hard-coding a fixed number of cycles, we compute cycles as a function of `N`, using `N + sqrt(N)`. That gives us roughly “about N cycles” for smaller portfolios, but grows more slowly as the problem size increases. We also clamp the result between a floor and a ceiling so the runtime stays predictable for demos. Separately, we set how many candidate swaps we test per cycle (`candidates_per_cycle`). More candidates per cycle gives better search coverage, but costs more runtime.

To keep the process explainable, we maintain a `history` list with one row per cycle. We also track `prev_assets`, which starts as the dataset's previous holdings (`previous_position == 1`). Why? Because the scoring function penalises turnover and transaction costs relative to a reference portfolio. In the first cycle, the reference is the real previous holdings from the dataset. After that, we update `prev_assets` to the best portfolio found in the last cycle, so each cycle evaluates candidates relative to the current best state.

The main loop runs through the chosen number of cycles. At the start of each cycle, we treat the current portfolio as the best portfolio so far and score it using the same scoring function from Cell 3. We then generate many candidate portfolios by proposing random swaps and scoring each one. We apply a greedy rule: we only accept a candidate if its total score is strictly better than the current best. By the end of the cycle, we keep the best portfolio we found during that cycle, record the key metrics in `history`, and carry it forward into the next cycle.

After the optimization finishes, we build a cycle history table and display it so we can see how the score, expected return, risk proxy, transaction cost, and number of changes evolved over time. We then build the final portfolio table by filtering the dataset down to the final asset list and sorting it by expected return. The printed labels explain what each column represents and how it relates back to the scoring model.

Finally, we print a small set of summary metrics for the final portfolio: sector balance, total expected return, and average market cap. We then stop the timer and print the runtime for Cell 4.




In [18]:
# ==============================
# Cell 4: Run optimization cycles and output the final portfolio
# ==============================
#
# What this cell does:
# 1) Starts from the "best start" portfolio chosen in Cell 3 (current_assets).
# 2) Runs a repeated improvement loop ("cycles") where each cycle proposes many simple swaps:
#       - remove 1 held asset
#       - add 1 not-held asset
#    and keeps the best candidate found in that cycle.
# 3) Records a cycle-by-cycle history so we can see what changed and why.
# 4) Prints the final portfolio and simple summary metrics.
#
# Important design choice:
# - The number of cycles is derived from N (the target portfolio size), not hard-coded.
# - This makes the run length scale up sensibly as the problem size increases.

start_time = time.perf_counter()

# ---- randomness control ----
# If use_fixed_seed is True, runs are reproducible (good for debugging).
# If use_fixed_seed is False, each run explores differently (good for exploration).
seed = config["random_seed"] if config.get("use_fixed_seed", False) else None
rng = np.random.default_rng(seed)

# ---- helper: propose one simple change (one-out, one-in) ----
def propose_swap(df, assets):
    """
    Takes the current portfolio (assets) and returns a new candidate portfolio
    with exactly one swap:
      - remove one currently-held asset
      - add one asset that is currently not held
    This keeps portfolio size constant at N.
    """
    held = list(assets)  # current holdings
    not_held = df.loc[~df["asset"].isin(held), "asset"].tolist()  # everything we do not hold

    # Safety guard: if we cannot swap, return unchanged.
    if not held or not not_held:
        return held

    # Choose one asset to sell and one to buy.
    out_a = rng.choice(held)
    in_a = rng.choice(not_held)

    # Build the new portfolio list.
    new_assets = held.copy()
    new_assets.remove(out_a)
    new_assets.append(in_a)

    return new_assets

# ---- compute number of cycles as a function of N ----
# Goal:
# - For small N, run about N cycles (a decent amount of exploration).
# - For larger N, grow slower than N so the run doesn't explode.
#
# Simple rule: cycles = N + sqrt(N)
#
# Then apply a floor and ceiling so demos don't take too long.

N = int(config["n_target"])

cycle_floor = int(config.get("cycle_floor", 6)) # minimum cycles even when N is tiny
cycle_ceiling = int(config.get("cycle_ceiling", 20)) # maximum cycles so runtime stays predictable

# Derived cycles (not hard-coded)
derived_cycles = int(round(N + math.sqrt(N)))

# Final cycles used
cycles = int(np.clip(derived_cycles, cycle_floor, cycle_ceiling))

# ---- how many candidate swaps we test per cycle ----
# More candidates = better chance of improvement, but slower runtime.
# Default is tied to N above (originally defined in Cell 2's config block).

cand_floor = int(config.get("cand_floor", 120))
cand_ceiling = int(config.get("cand_ceiling", 800))

derived_candidates = int(round(8 * N))  # 8 swap ideas per slot
candidates_per_cycle = int(np.clip(derived_candidates, cand_floor, cand_ceiling))

# Comment out everything from 'cand_floor' to 'candidates_per_cycle' above if you'd prefer to use a fixed default
# candidates_per_cycle = int(config.get("candidates_per_cycle", 300))

# ---- storage for explainability ----
# We store a row per cycle so we can show how things evolved over time.

history = []

# ---- previous holdings ----
# This is the "anchor" portfolio we compare against for turnover and transaction costs.
# In cycle 1, we compare our start portfolio against the dataset's previous_position.
# After each cycle, we set prev_assets = current best portfolio for the next cycle.

prev_assets = df.loc[df["previous_position"] == 1, "asset"].tolist()

# ---- main optimization loop ----
# Each "cycle" is one improvement step.
# Inside each cycle we:
# - start with the current portfolio as "best"
# - try many random swaps
# - keep the best scoring portfolio found

for cycle in range(1, cycles + 1):

    # Current best portfolio at the start of this cycle
    best_assets = list(current_assets)

    # Score it using the scoring model from Cell 3
    best_score = score_start(df, best_assets, prev_assets, score_config)

    # Try many candidate swaps and keep the best one found
    for _ in range(candidates_per_cycle):
        cand_assets = propose_swap(df, best_assets)
        cand_score = score_start(df, cand_assets, prev_assets, score_config)

        # Greedy accept: only move if strictly better
        if cand_score["total_score"] > best_score["total_score"]:
            best_assets = cand_assets
            best_score = cand_score

    # Record cycle summary so we can show progress
    history.append({
        "cycle": cycle,
        "total_score": round(best_score["total_score"], 4),
        "expected_return": round(best_score["total_expected_return"], 4),
        "risk_proxy": round(best_score["risk_proxy"], 6),
        "transaction_cost": round(best_score["transaction_cost"], 4),
        "num_changes": int(best_score["num_changes"]),
        "sectors": best_score["sector_counts"],
    })

    # Move forward: the best portfolio found becomes the new current portfolio
    prev_assets = best_assets
    current_assets = best_assets

# ---- cycle history table (with labels) ----
history_df = pd.DataFrame(history)

print("Cycle history")
print(
    f"This table shows how the portfolio evolved over {cycles} cycles. "
    f"Each cycle tries {candidates_per_cycle} swap ideas, then keeps the best result found.\n"
    f"Cycle rule: cycles = clip(N + sqrt(N), floor={cycle_floor}, ceiling={cycle_ceiling}). "
    f"With N={N}, cycles={cycles}.\n"
)

display(history_df)

# ---- final portfolio table (with labels) ----
final_assets = sorted(current_assets)

final_df = (
    df[df["asset"].isin(final_assets)]
    .sort_values("expected_return", ascending=False)
)

print("Final portfolio")
print(
    "This table shows the final N assets after the optimization run, sorted by expected return.\n"
    "Columns:\n"
    "- expected_return: the dataset’s return signal (higher is better)\n"
    "- market_cap: used as a simple risk proxy (bigger is treated as safer)\n"
    "- transaction_cost: cost paid when we change this asset (buy or sell)\n"
    "- previous_position: whether it was held at the start (1) or not (0)\n"
)

display(final_df[[
    "asset",
    "sector",
    "expected_return",
    "market_cap",
    "transaction_cost",
    "previous_position",
]])

# ---- final summary stats (with labels) ----
print("Final summary")
print("These are simple, explainable summary metrics for the final portfolio.\n")

print("Final sector balance:", final_df["sector"].value_counts().to_dict())
print("Total expected return:", round(final_df["expected_return"].sum(), 4))
print("Average market cap:", round(final_df["market_cap"].mean(), 2))

# ---- TIMING ----
elapsed = time.perf_counter() - start_time
print(f"\nCell 4 runtime: {elapsed:.3f} seconds\n")


Cycle history
This table shows how the portfolio evolved over 20 cycles. Each cycle tries 320 swap ideas, then keeps the best result found.
Cycle rule: cycles = clip(N + sqrt(N), floor=6, ceiling=20). With N=40, cycles=20.



,cycle,total_score,expected_return,risk_proxy,transaction_cost,num_changes,sectors
0,1,-436.4314,14.1323,0.283766,0.3367,10,"{'ENERGY': 12, 'HEALTH': 8, 'CONS': 8, 'FIN': ..."
1,2,-434.1003,16.4502,0.281200,0.3255,10,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
2,3,-433.7748,16.4502,0.281200,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
3,4,-433.7578,16.5197,0.280619,0.0529,2,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
4,5,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
5,6,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
6,7,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
7,8,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
8,9,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."
9,10,-433.7048,16.5197,0.280619,0.0000,0,"{'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'TECH':..."


Final portfolio
This table shows the final N assets after the optimization run, sorted by expected return.
Columns:
- expected_return: the dataset’s return signal (higher is better)
- market_cap: used as a simple risk proxy (bigger is treated as safer)
- transaction_cost: cost paid when we change this asset (buy or sell)
- previous_position: whether it was held at the start (1) or not (0)



,asset,sector,expected_return,market_cap,transaction_cost,previous_position
4,ASSET_019,ENERGY,1.528005,16.221097,0.040161,0
30,ASSET_021,CONS,1.020886,8.424146,0.025789,1
12,ASSET_043,FIN,0.816748,3.107359,0.011257,0
11,ASSET_042,ENERGY,0.806086,14.331803,0.047954,0
23,ASSET_013,FIN,0.800446,2.053882,0.038786,1
27,ASSET_017,CONS,0.794313,50.350114,0.035909,1
9,ASSET_039,FIN,0.782384,28.469388,0.046443,0
16,ASSET_002,HEALTH,0.765114,5.646328,0.025405,1
47,ASSET_047,CONS,0.734740,17.544573,0.020667,1
6,ASSET_032,ENERGY,0.665292,8.781033,0.036675,0


Final summary
These are simple, explainable summary metrics for the final portfolio.

Final sector balance: {'ENERGY': 12, 'CONS': 9, 'HEALTH': 7, 'FIN': 6, 'TECH': 6}
Total expected return: 16.5197
Average market cap: 15.54

Cell 4 runtime: 28.547 seconds



## Qiskit checks

In [19]:
# ==============================
# Qiskit check / conditional install
# ==============================
#
# What this cell does:
# 1) Checks whether qiskit is already installed in the environment.
# 2) If NOT installed, installs qiskit, qiskit-algorithms, qiskit-optimization.
# 3) If installed, prints the installed versions.
#
# This avoids reinstalling Qiskit every run and prevents long stalls in Cell 5.

start_time = time.perf_counter()

import importlib.util
import subprocess
import sys

def is_installed(pkg_name):
    return importlib.util.find_spec(pkg_name) is not None

packages = [
    "qiskit",
    "qiskit_algorithms",
    "qiskit_optimization",
    "qiskit_aer",
]

missing = [p for p in packages if not is_installed(p)]

if missing:
    print("Qiskit not fully installed. Installing missing packages:")
    print(missing)
    subprocess.check_call([
        sys.executable,
        "-m",
        "pip",
        "install",
        "-q",
        "qiskit",
        "qiskit-algorithms",
        "qiskit-optimization",
        "qiskit-aer",
    ])
    print("Installation complete.\n")
else:
    print("Qiskit already installed.\n")

# Display versions (works whether just installed or already present)
import qiskit
import qiskit_algorithms
import qiskit_optimization
import qiskit_aer

print("Qiskit versions:")
print("qiskit:", qiskit.__version__)
print("qiskit-algorithms:", qiskit_algorithms.__version__)
print("qiskit-optimization:", qiskit_optimization.__version__)
print("qiskit-aer:", qiskit_aer.__version__)

# ---- TIMING ----
elapsed = time.perf_counter() - start_time
print(f"\nCell runtime: {elapsed:.3f} seconds")


Qiskit already installed.

Qiskit versions:
qiskit: 2.2.3
qiskit-algorithms: 0.4.0
qiskit-optimization: 0.7.0
qiskit-aer: 0.17.2

Cell runtime: 0.002 seconds


## Cell 5 — Quantum optimizations

Note: The current version has a Colab runtime of about 5 minutes. For shorter/faster demos, tweak the `quantum_config` block in the cell — see instructions within.

## What Cell 5 does (in soccer terms)

Think of the full dataset passed in Cell 1 as a huge pool of footballers in a country. We want to pick a starting 11 for the season, but we also want to compare two different "coaching approaches" on a small, controlled tryout before we scale up.

Cell 5 is that tryout. In soccer terms, we're trying to pick a **small matchday squad** from a shortlist of players, where:

* Each player is either **selected or not selected**.
* We must pick **exactly Nq players**.
* We want a squad that scores well on a combined "team quality" score:

  * Lots of attacking output (expected return).
  * Lower fragility/injury risk (market cap risk proxy).
  * Less disruption to the existing team (transaction cost + previous_position turnover proxy).

Then we solve the same selection problem using:

* A **classical coach** who can search perfectly (exact solver).
* A **quantum-style coach** using QAOA on a simulator (approximate solver).

Finally, we evaluate both squads using the existing `score_start()` metric so the results align with the rest of the notebook.

### How the variables map to the soccer analogy

#### The "league rules" and setup

* `df`: The full scouting database of all players, with stats and metadata.

* `config`: The broader season plan (how big the real squad should be, etc.). Cell 5 borrows from it but runs a smaller demo.

* `score_start`, `score_config`: The existing "team evaluation framework." After both coaches pick squads, we grade them using the same rubric we used earlier.

* `required_names`, `missing`: A pre-match checklist. If the scouting database or rules are missing, the match can't start.

* `start_time`, `elapsed`: The match clock. We track how long setup and the overall tryout took.

#### The demo controls (the “friendly match” settings)

* `quantum_config`: The friendly-match settings sheet. It controls the size of the tryout and how hard we search.

Key parts inside it:

* `quantum_config["n_qubits"]` → **Nq**, the number of players we must select for the demo squad. In soccer terms: "Pick exactly 6 players for this mini-scrimmage squad." (It's called `n_qubits` because each decision is a binary pick/no-pick.)

* `quantum_config["shortlist_min"]`, `quantum_config["shortlist_max"]` → bounds for **Mq**, the size of the shortlist. In soccer terms: "Only consider 6-10 players for this tryout."

* `quantum_config["shortlist_top_return"]` → how far down the 'goals + assists' leaderboard we scan when building the shortlist. In soccer terms: "Look at the top 10 attackers by output, then pick from there."

* `quantum_config["sector_cap"]` → the formation balance rule enforced during shortlist building. In soccer terms: "Don't shortlist too many players from the same position group." Sector is like the position group in soccer (DEF/MID/FWD), and the cap stops us from building a shortlist that is basically all strikers.

* `quantum_config["w_return"]`, `quantum_config["w_risk"]`, `quantum_config["w_turnover"]` → how the coach values different things:

  * `w_return`: preference for match-winning output.
  * `w_risk`: penalty for fragility (injury-prone / low "reliability"), using our proxy.
  * `w_turnover`: penalty for tearing up the existing squad too much with swaps/transfers.

* `quantum_config["qaoa_reps"]` → how many "tactical layers" the quantum-style coach is allowed. More layers means richer tactics, but more thinking time.

* `quantum_config["qaoa_maxiter"]` → how long the quantum-style coach is allowed to tweak tactics in training (i.e., how many optimisation steps).

* `quantum_config["qubo_var_ceiling"]` → a safety limit on complexity. In soccer terms: "If the training drill becomes too complex, we cancel the session before it eats the whole day."

* `quantum_config["run_exact_if_qubo_vars_leq"]` → a rule for whether we let the classical coach do a perfect search. If the shortlist is too big, we skip perfect search as it becomes too slow.

* `quantum_config["shots"]` is a quantum sampling setting. In this specific version of the cell, it doesn't matter because we're using `StatevectorSampler` (no shots). If you later switch back to Aer, it becomes "how many practice scrimmages we run to estimate performance."

#### The random seed

* `seed`: Whether we want the tryout to be repeatable. In soccer terms: "Do we want the same training drill every time?"
* `rng`: The random number generator. In this cell it's created but not actually used later, because the current demo isn't doing random swaps here. It's leftover scaffolding for reproducibility.

#### Squad size and shortlist size

* `N_full`: The real squad target from the main problem (the full season plan).
* `Nq`: The demo squad size for the quantum tryout.
* `Mq_derived`: A heuristic target for how big the tryout shortlist should be (2*Nq + 5).
* `Mq`: The final shortlist size after applying min/max and forcing it to be bigger than Nq.

In soccer terms:

* N_full = "Our real season squad size."
* Nq = "How many players we pick in this drill."
* Mq = "How many players we invite to the drill."

#### Building the shortlist (who gets invited to the tryout)

* `prev_assets`: The players currently in the team (previous holdings).

* `build_shortlist_with_sector_cap(...)`: The scout who invites players to the tryout:

  * `scan_top_k`: how many top performers we look at first.
  * `counts`: how many players we've already invited per position group.
  * `shortlist`: the invite list.
  * `df_sorted`, `remainder`: the ranked scouting lists used to fill the shortlist.

* `shortlist`: The final invited players.

* `shortlist_df`: The printed roster of invited players with their stats.

#### Building the “pick squad” decision model

* `qp`: The coach's selection sheet. It contains all the yes/no decisions plus the rules and the objective.

* `a`: A player ID.

* `qp.binary_var(name=a)`: A checkbox: pick this player (1) or don't (0).

* `linear_obj`: The coach's scoring model for each player, the per-player contribution to the objective.

Per-player stats:

* `er` (expected_return): "How much match output we expect from this player."

* `cap` (market_cap): "How safe / reliable we treat this player." Bigger is safer in our proxy.

* `risk_i = 1/cap`: "Fragility risk." Bigger risk means worse.

* `tc` (transaction_cost): "Disruption cost if we change this player in or out."

* `prev_i` (previous_position): "Was this player already in the current squad?"

* `turnover_coeff`: The logic that rewards continuity and penalises churn. In soccer terms:

  * If the player is already in the squad, keeping them is treated as less disruptive (even slightly rewarded).
  * If they aren't, adding them is treated as disruptive (penalized).

* `qp.maximize(linear=linear_obj)`: "Pick the squad that maximises team quality."

Constraint:

* `qp.linear_constraint(... rhs=Nq ...)`: "Pick exactly Nq players." In soccer terms: "You must name exactly 6 players."

#### Converting to QUBO and solving (two coaches)

* `qubo`: The same selection problem converted into a format QAOA can handle.
* `n_asset_vars`: How many invitees (checkboxes).
* `n_qubo_vars`: How many effective binary variables exist after conversion.
* `quantum_config["qubo_var_ceiling"]`: The 'too complex' tripwire from earlier.

Solvers:

* `exact_solver`, `exact_result`: The classical coach doing an exhaustive, perfect selection (only if allowed).
* `sampler`, `sampler_label`: The training environment for the quantum-style coach. Here we chose:

  * `StatevectorSampler`: a perfect, noiseless simulator for the drill.
* `qaoa`, `qaoa_solver`, `qaoa_result`: The quantum-style coach who uses QAOA to search for a good squad.

#### Translating solver outputs into actual squads

* `asset_set`: The invited players.
* `result_to_assets_from_qubo(...)`: Turns a binary vector of checkboxes into an actual player list.
* `exact_assets`: The classical coach's chosen squad.
* `qaoa_assets`: The quantum coach's chosen squad.

#### Scoring and presenting results

* `exact_score`, `qaoa_score`: The existing "team performance report" applied to each chosen squad.
* `rows`, `comparison`: The match report table comparing the coaches.
* The final printed tables are the two squads with their stats.

---

In summary: we run a small “team selection tryout” on a shortlist, under balance rules, and compare a perfect classical picker with a quantum-inspired picker, then score both squads using the same scoring system used earlier so the results remain comparable.


In [20]:
# ==============================
# Cell 5 (revamped): Quantum step (fast demo, shortlist-capped, QAOA on simulator)
# ==============================
#
# Design goals for this version:
# - Keep runtime predictable (seconds to a couple minutes in Colab).
# - Ensure the “10-qubit demo” is actually a small problem after QUBO conversion.
# - Avoid sector-cap constraints inside the QUBO (they add extra binaries).
#   Instead, we enforce sector caps when building the shortlist so the cap is “baked in.”

start_time = time.perf_counter()

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from qiskit_algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA

# The code might throw a SparseEfficiency warning that doesn't stop runtime — this bit filters out that warning.
import warnings
from scipy.sparse import SparseEfficiencyWarning
warnings.filterwarnings("ignore", category=SparseEfficiencyWarning)


# ------------------------------------------------------------
# 0) Preconditions (fail fast if the kernel restarted before running this cell)
# ------------------------------------------------------------
required_names = ["df", "config", "score_start", "score_config"]
missing = [n for n in required_names if n not in globals()]
if missing:
    raise NameError(
        "Missing required objects in memory: "
        + ", ".join(missing)
        + ". Run Cells 1–4 again, then re-run this cell."
    )

# ------------------------------------------------------------
# 1) Quantum demo knobs — Tweak these to test different outcomes/runtimes
# ------------------------------------------------------------
quantum_config = {
    # Target selection size for the quantum demo (Mq)
    # Keep this between 5 to 8 to avoid a long runtime; change to 3 or less for a faster run
    "n_qubits": 6,

    # Shortlist size (effectively Mq) must be greater than Nq above so there is an actual choice
    # We start off with tiny numbers for a shorter demo — tweak as needed
    "shortlist_min": 6,
    "shortlist_max": 10,

    # Shortlist construction weights
    "shortlist_top_return": 10,   # we scan the top K by return and select under sector cap
    "shortlist_top_cap": 0,
    "shortlist_prev": 0,

    # Objective weights inside the optimization model (linear)
    "w_return": 1.0,
    "w_risk": 0.5,        # penalty for 1/market_cap
    "w_turnover": 1.0,    # proxy using transaction_cost + previous_position

    # Sector cap: enforced in shortlist building (not as a QUBO constraint)
    #"sector_cap": int(config.get("sector_cap", 5)),
    "sector_cap": 2,        # keep small so the shortlist fills easily

    # QAOA settings (kept tight for speed)
    # Drop the number of reps and max iterations (maxiter) if runtime is too long
    "qaoa_reps": 1,
    "qaoa_maxiter": 6,
    "shots": 100,

    # Safety guard: if QUBO expands too much, stop early
    # If you're getting early errors, try increasing this number
    "qubo_var_ceiling": 15,

    # Classical exact baseline guard (exact becomes exponential quickly)
    # if this number is less than qubo_var_ceiling above, it won't run the classical baseline
    "run_exact_if_qubo_vars_leq": 20,
}

# Seed for reproducibility (ties to the existing config flag)
seed = int(config.get("random_seed", 42)) if config.get("use_fixed_seed", False) else None
rng = np.random.default_rng(seed) # random number generator

# ------------------------------------------------------------
# 2) Build shortlist (Mq assets) and set quantum target Nq
# ------------------------------------------------------------
N_full = int(config["n_target"])
Nq = int(min(N_full, quantum_config["n_qubits"]))  # quantum demo target (e.g., 10)

Mq_derived = int(2 * Nq + 5)
Mq = int(np.clip(Mq_derived, quantum_config["shortlist_min"], quantum_config["shortlist_max"]))
Mq = max(Mq, Nq + 1)  # force Mq > Nq

prev_assets = df.loc[df["previous_position"] == 1, "asset"].tolist()

def build_shortlist_with_sector_cap(df, Mq, sector_cap, scan_top_k=60):
    """
    Build a shortlist by scanning top expected_return assets and taking them
    only if their sector has not hit the sector_cap yet.
    Result: each sector appears at most sector_cap times in the shortlist.
    """
    df_sorted = df.sort_values("expected_return", ascending=False).head(scan_top_k)
    counts = {}
    shortlist = []

    for _, row in df_sorted.iterrows():
        a = row["asset"]
        s = row["sector"]
        if counts.get(s, 0) >= sector_cap:
            continue
        shortlist.append(a)
        counts[s] = counts.get(s, 0) + 1
        if len(shortlist) >= Mq:
            break

    # If we still did not fill (rare), fill with next-best assets but keep the cap.
    if len(shortlist) < Mq:
        remainder = df.sort_values("expected_return", ascending=False)
        for _, row in remainder.iterrows():
            a = row["asset"]
            if a in shortlist:
                continue
            s = row["sector"]
            if counts.get(s, 0) >= sector_cap:
                continue
            shortlist.append(a)
            counts[s] = counts.get(s, 0) + 1
            if len(shortlist) >= Mq:
                break

    return shortlist[:Mq]

shortlist = build_shortlist_with_sector_cap(
    df=df,
    Mq=Mq,
    sector_cap=int(quantum_config["sector_cap"]),
    scan_top_k=int(quantum_config["shortlist_top_return"]),
)

print("Quantum shortlist (fast demo)")
print(f"Full target N_full={N_full}; quantum target Nq={Nq}; shortlist size Mq={len(shortlist)}")
print(f"Sector cap (enforced in shortlist): {quantum_config['sector_cap']}")
print("")

shortlist_df = (
    df[df["asset"].isin(shortlist)][
        ["asset", "sector", "expected_return", "market_cap", "transaction_cost", "previous_position"]
    ]
    .sort_values("expected_return", ascending=False)
    .reset_index(drop=True)
)
display(shortlist_df)

# ------------------------------------------------------------
# 3) Build the binary optimization model (no sector constraints inside QUBO)
# ------------------------------------------------------------
qp = QuadraticProgram(name="portfolio_selection_shortlist_fast")
for a in shortlist:
    qp.binary_var(name=a)

linear_obj = {}
for a in shortlist:
    row = df.loc[df["asset"] == a].iloc[0]
    er = float(row["expected_return"])
    cap = float(row["market_cap"])
    tc = float(row["transaction_cost"])
    prev_i = int(row["previous_position"])

    risk_i = 1.0 / max(cap, 1e-6)

    # Turnover proxy:
    # prev=0 -> +tc when selected (penalize adding new names)
    # prev=1 -> -tc when selected (reward keeping old names)
    # To nix or equalize this effect, change turnover_coeff to 0 or tc respectively
    turnover_coeff = (1 - 2 * prev_i) * tc

    linear_obj[a] = (
        quantum_config["w_return"] * er
        - quantum_config["w_risk"] * risk_i
        - quantum_config["w_turnover"] * turnover_coeff
    )

qp.maximize(linear=linear_obj)

# Only constraint we keep in-model: pick exactly Nq assets
qp.linear_constraint(
    linear={a: 1 for a in shortlist},
    sense="==",
    rhs=Nq,
    name="choose_exactly_Nq"
)

print("Optimization model built")
print(f"- Decision variables (assets): {len(shortlist)}")
print(f"- Pick exactly Nq = {Nq}")
print("")

elapsed = time.perf_counter() - start_time
print(f"Runtime to build model: {elapsed:.3f} seconds\n")

# ------------------------------------------------------------
# 4) Convert to QUBO and solve (exact vs QAOA) with guards
# ------------------------------------------------------------
qubo = QuadraticProgramToQubo().convert(qp)

n_asset_vars = len(shortlist)
n_qubo_vars = qubo.get_num_binary_vars()

print(f"Original binaries (assets): {n_asset_vars}")
print(f"QUBO binaries (after conversion): {n_qubo_vars}")

if n_qubo_vars > int(quantum_config["qubo_var_ceiling"]):
    raise ValueError(
        f"QUBO expanded to {n_qubo_vars} binaries (ceiling={quantum_config['qubo_var_ceiling']}). "
        f"Reduce shortlist_max/min or simplify constraints."
    )

# Sampler choice: shot-based Aer or StatevectorSampler.

sampler = None
sampler_label = None

# NOTE: Aer can take a while, so we'll eschew it for demo purposes.
# To test it out later, uncomment the following lines and comment out the Statevector snippet below

'''
try:
    from qiskit_aer.primitives import Sampler as AerSampler
    sampler = AerSampler(run_options={"shots": int(quantum_config["shots"])}, seed=seed)
    sampler_label = f"AerSampler (shots={quantum_config['shots']})"
except Exception as e:
    print("Aer sampler unavailable, falling back to StatevectorSampler.")
    print("Reason:", repr(e))
    from qiskit.primitives import StatevectorSampler
    sampler = StatevectorSampler(seed=seed)
    sampler_label = "StatevectorSampler"
'''

# For demo purposes, we'll stick with Statevector
# For Aer, comment out the following three lines and uncomment the snippet above

from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler(seed=seed)
sampler_label = "StatevectorSampler"

print(f"Sampler: {sampler_label}\n")

# QAOA solver
qaoa = QAOA(
    sampler=sampler,
    optimizer=COBYLA(maxiter=int(quantum_config["qaoa_maxiter"])),
    reps=int(quantum_config["qaoa_reps"]),
)
qaoa_solver = MinimumEigenOptimizer(qaoa)

# Optional exact baseline (only when small enough)
exact_result = None
if n_qubo_vars <= int(quantum_config["run_exact_if_qubo_vars_leq"]):
    exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
    exact_result = exact_solver.solve(qubo)
else:
    print(f"Skipping exact baseline (QUBO has {n_qubo_vars} binaries)\n")

qaoa_result = qaoa_solver.solve(qubo)

def result_to_assets_from_qubo(result, asset_set):
    """
    Convert a QUBO solution vector into chosen asset names.
    We only read variables whose names match assets (ignore any ancillas).
    """
    chosen = []
    for var, v in zip(qubo.variables, result.x):
        name = var.name
        if name in asset_set and int(round(v)) == 1:
            chosen.append(name)
    return chosen

asset_set = set(shortlist)

exact_assets = []
if exact_result is not None:
    exact_assets = result_to_assets_from_qubo(exact_result, asset_set)

qaoa_assets = result_to_assets_from_qubo(qaoa_result, asset_set)

# ------------------------------------------------------------
# 5) Score and present results using our existing score_start()
# ------------------------------------------------------------
rows = []

if exact_result is not None:
    exact_score = score_start(df, exact_assets, prev_assets, score_config)
    rows.append({
        "solver": "Classical exact (baseline)",
        "num_assets": len(exact_assets),
        "total_score": round(exact_score["total_score"], 4),
        "expected_return": round(exact_score["total_expected_return"], 4),
        "risk_proxy": round(exact_score["risk_proxy"], 6),
        "transaction_cost": round(exact_score["transaction_cost"], 4),
        "num_changes": int(exact_score["num_changes"]),
        "sectors": ", ".join(f"{k}:{v}" for k, v in exact_score["sector_counts"].items()),
    })

qaoa_score = score_start(df, qaoa_assets, prev_assets, score_config)
rows.append({
    "solver": f"QAOA (simulator, Nq={Nq}, Mq={len(shortlist)})",
    "num_assets": len(qaoa_assets),
    "total_score": round(qaoa_score["total_score"], 4),
    "expected_return": round(qaoa_score["total_expected_return"], 4),
    "risk_proxy": round(qaoa_score["risk_proxy"], 6),
    "transaction_cost": round(qaoa_score["transaction_cost"], 4),
    "num_changes": int(qaoa_score["num_changes"]),
    "sectors": ", ".join(f"{k}:{v}" for k, v in qaoa_score["sector_counts"].items()),
})

comparison = pd.DataFrame(rows)

print("\nSolver comparison (scored with score_start)")
display(comparison)

print("Chosen assets")
if exact_result is not None:
    print("Classical exact:", sorted(exact_assets))
print("QAOA:", sorted(qaoa_assets))

print("\nDetailed portfolios")

if exact_result is not None:
    print("Classical exact portfolio")
    display(
        df[df["asset"].isin(exact_assets)][
            ["asset", "sector", "expected_return", "market_cap", "transaction_cost", "previous_position"]
        ]
        .sort_values("expected_return", ascending=False)
        .reset_index(drop=True)
    )

print("QAOA portfolio")
display(
    df[df["asset"].isin(qaoa_assets)][
        ["asset", "sector", "expected_return", "market_cap", "transaction_cost", "previous_position"]
    ]
    .sort_values("expected_return", ascending=False)
    .reset_index(drop=True)
)

elapsed = time.perf_counter() - start_time
print(f"\nCell 5 runtime (total): {elapsed:.3f} seconds")


Quantum shortlist (fast demo)
Full target N_full=40; quantum target Nq=6; shortlist size Mq=10
Sector cap (enforced in shortlist): 2



,asset,sector,expected_return,market_cap,transaction_cost,previous_position
0,ASSET_019,ENERGY,1.528005,16.221097,0.040161,0
1,ASSET_021,CONS,1.020886,8.424146,0.025789,1
2,ASSET_043,FIN,0.816748,3.107359,0.011257,0
3,ASSET_042,ENERGY,0.806086,14.331803,0.047954,0
4,ASSET_013,FIN,0.800446,2.053882,0.038786,1
5,ASSET_017,CONS,0.794313,50.350114,0.035909,1
6,ASSET_002,HEALTH,0.765114,5.646328,0.025405,1
7,ASSET_018,TECH,0.592766,48.538721,0.041850,1
8,ASSET_025,HEALTH,0.560751,23.411763,0.048984,1
9,ASSET_033,TECH,0.538884,4.745871,0.042932,0


Optimization model built
- Decision variables (assets): 10
- Pick exactly Nq = 6

Runtime to build model: 0.033 seconds

Original binaries (assets): 10
QUBO binaries (after conversion): 10
Sampler: StatevectorSampler


Solver comparison (scored with score_start)


,solver,num_assets,total_score,expected_return,risk_proxy,transaction_cost,num_changes,sectors
0,Classical exact (baseline),6,-1685.6075,5.7312,0.338952,1.0675,36,"ENERGY:2, CONS:2, FIN:1, HEALTH:1"
1,"QAOA (simulator, Nq=6, Mq=10)",6,-1686.1546,5.2197,0.360217,1.0861,36,"ENERGY:2, HEALTH:2, TECH:1, CONS:1"


Chosen assets
Classical exact: ['ASSET_002', 'ASSET_017', 'ASSET_019', 'ASSET_021', 'ASSET_042', 'ASSET_043']
QAOA: ['ASSET_002', 'ASSET_019', 'ASSET_021', 'ASSET_025', 'ASSET_033', 'ASSET_042']

Detailed portfolios
Classical exact portfolio


,asset,sector,expected_return,market_cap,transaction_cost,previous_position
0,ASSET_019,ENERGY,1.528005,16.221097,0.040161,0
1,ASSET_021,CONS,1.020886,8.424146,0.025789,1
2,ASSET_043,FIN,0.816748,3.107359,0.011257,0
3,ASSET_042,ENERGY,0.806086,14.331803,0.047954,0
4,ASSET_017,CONS,0.794313,50.350114,0.035909,1
5,ASSET_002,HEALTH,0.765114,5.646328,0.025405,1


QAOA portfolio


,asset,sector,expected_return,market_cap,transaction_cost,previous_position
0,ASSET_019,ENERGY,1.528005,16.221097,0.040161,0
1,ASSET_021,CONS,1.020886,8.424146,0.025789,1
2,ASSET_042,ENERGY,0.806086,14.331803,0.047954,0
3,ASSET_002,HEALTH,0.765114,5.646328,0.025405,1
4,ASSET_025,HEALTH,0.560751,23.411763,0.048984,1
5,ASSET_033,TECH,0.538884,4.745871,0.042932,0



Cell 5 runtime (total): 242.510 seconds
